# Day 1

## Part 1

At first I tried to use `map read` on the lines of the input, but that generates a runtime error. Positive frequencies are encoded with a leading `+` sign and Haskell refuses to parse those as one might expect. So here, we define a custom `parseInt` function that pulls off the leading `+` signs before passing the rest onto `read`.

In [1]:
parseInt :: String -> Int
parseInt ('+':rest) = read rest
parseInt str = read str

With that out of the way, we compute the resulting frequency after passing through all of the additions and subtracts by just using the `sum` function on the parsed input.

In [2]:
resultingFrequency :: String -> Int
resultingFrequency = sum . map parseInt . lines

In [3]:
readFile "./inputs/1.txt" >>= print . resultingFrequency

500

## Part 2

In [4]:
import qualified Data.Set as Set

intermediateFrequencies :: [Int] -> [Int]
intermediateFrequencies xs = 0 : f (0, xs)
  where
    f (currentFreq, []) = []
    f (currentFreq, (x:xs)) = let newFreq = currentFreq + x in newFreq : f (newFreq, xs)

firstDuplicate :: (Ord a) => [a] -> a
firstDuplicate xs = f Set.empty xs
  where
    f visited (x:xs) = if x `Set.member` visited
                         then x
                         else f (Set.insert x visited) xs

firstDuplicateFrequency :: String -> Int
firstDuplicateFrequency = firstDuplicate . intermediateFrequencies . cycle . map parseInt . lines

In [5]:
readFile "./inputs/1.txt" >>= print . firstDuplicateFrequency

709

# Day 2

## Part 1

There's a function `group` in `Data.List` that groups together subsequences of equal elements in a list.

In [6]:
import Data.List (group)
group "Mississippi"

["M","i","ss","i","ss","i","pp","i"]

If we apply this to a box ID after we sort its letters we'll get something like this.

In [7]:
let boxId = "evsialkqydurohxqpwbcugtjmh"

import Data.List (sort)
group . sort $ boxId

["a","b","c","d","e","g","hh","i","j","k","l","m","o","p","qq","r","s","t","uu","v","w","x","y"]

Now we can just test if there are any elements of this list that contain exactly 2 characters and test if there are any that contain exactly 3 elements.

In [8]:
any (\x -> length x == 2) . group . sort $ boxId

True

In [9]:
any (\x -> length x == 3) . group . sort $ boxId

False

Putting it all together:

In [10]:
checksum :: [String] -> Int
checksum xs = numIdsWithRepeatedChars 2 * numIdsWithRepeatedChars 3
  where
    numIdsWithRepeatedChars n = length . filter (any (\x -> length x == n) . group . sort) $ xs

In [11]:
day2Input <- readFile "./inputs/2.txt"
checksum . lines $ day2Input

6448

## Part 2

Naively, we can test each string against every other string to count the number of differing characters. I want a function that, given a list, returns a list of pairs of elements.

In [12]:
pairs :: [a] -> [(a,a)]
pairs (x:xs) = map (\y -> (x, y)) xs ++ pairs xs
pairs [] = []

For example,

In [13]:
pairs [1,2,3,4]

[(1,2),(1,3),(1,4),(2,3),(2,4),(3,4)]

Now that we can compute all of the pairs on box IDs we need to compare, we a way to calculate if two box IDs differ by exactly one character.

In [14]:
editDistance :: Eq a => [a] -> [a] -> Int
editDistance xs ys = length . filter (\(x, y) -> x /= y) $ zip xs ys

In [15]:
editDistance "abc" "xyz"

3

In [16]:
editDistance "abc" "azc"

1

Now we can find the unique pair of box IDs whose `editDistance` is 1. But after that, we'll need to construct the string composed of the letters shared between the two.

In [17]:
sharedSubsequence :: Eq a => [a] -> [a] -> [a]
sharedSubsequence xs ys = map fst . filter (\(x, y) -> x == y) $ zip xs ys

In [18]:
sharedSubsequence "abcde" "abzde"

"abde"

Finally,

In [19]:
import Data.List (find)

commonLettersBetweenCorrectBoxIds :: [String] -> Maybe String
commonLettersBetweenCorrectBoxIds xs = uncurry sharedSubsequence <$> pair
  where
    pair = find ((== 1) . uncurry editDistance) . pairs $ xs

In [20]:
commonLettersBetweenCorrectBoxIds . lines $ day2Input

Just "evsialkqyiurohzpwucngttmf"

# Day 3

## Part 1

The first challenge is parsing the input format. The input contains "claims" formatted like this:

```
#1 @ 662,777: 18x27
#2 @ 893,985: 13x10
#3 @ 199,328: 16x16
...
```

For Advent of Code projects in the past, I've used parser combinator libraries Parsec and Attoparsec for parsing even simple things like this. I think this time I'll just try to use a custom `Read` instance.

In [21]:
data FabricClaim = FabricClaim { identifier :: Int
                               , position :: (Int, Int)
                               , size :: (Int, Int) } deriving (Show, Eq)

In [22]:
import Data.Char (isDigit)

instance Read FabricClaim where
  readsPrec _ input =
    let ('#':rest1) = input
        (claimId, rest2) = span isDigit rest1
        (' ' : '@' : ' ' : rest3) = rest2
        (positionX, rest4) = span isDigit rest3
        (',':rest5) = rest4
        (positionY, rest6) = span isDigit rest5
        (':' : ' ' : rest7) = rest6
        (width, rest8) = span isDigit rest7
        ('x' : rest9) = rest8
        (height, rest10) = span isDigit rest9
        in
    [(FabricClaim (read claimId) (read positionX, read positionY) (read width, read height), rest10)]

In [23]:
read "#1 @ 662,777: 18x27" :: FabricClaim

FabricClaim {identifier = 1, position = (662,777), size = (18,27)}

In [24]:
read "[#1 @ 662,777: 18x27,#2 @ 893,985: 13x10]" :: [FabricClaim]

[FabricClaim {identifier = 1, position = (662,777), size = (18,27)},FabricClaim {identifier = 2, position = (893,985), size = (13,10)}]

I want to fold over the list of claims. I'll need to keep track of the total area claimed so far by at least one elf as well as the total area claimed by more than one elf. Naturally, I'll want to convert every claim to a set of coordinates.

In [25]:
getCoords :: FabricClaim -> Set.Set (Int, Int)
getCoords (FabricClaim claimdId (startX, startY) (w, h)) =
  Set.fromList [(x, y) | x <- [startX..(startX + w - 1)],
                         y <- [startY..(startY + h - 1)]]

In [26]:
getCoords $ FabricClaim 0 (1,2) (3,4)

fromList [(1,2),(1,3),(1,4),(1,5),(2,2),(2,3),(2,4),(2,5),(3,2),(3,3),(3,4),(3,5)]

Thus,

In [27]:
import Data.List (foldl')

areaInContention :: [FabricClaim] -> Set.Set (Int, Int)
areaInContention claims =
  let f (contended, claimed) claim = (Set.union contended . Set.intersection claimed $ getCoords claim,
                                      Set.union claimed $ getCoords claim)
    in
      fst $ foldl' f (Set.empty, Set.empty) claims               

In [28]:
day3Input <- readFile "./inputs/3.txt"
print . length . areaInContention . map read . lines $ day3Input

120408

## Part 2

Now we want to find the unique claim that intersects no other claim. For this, we can iterate over every claim and check to see if its coordinate set is disjoint with the union of the coordinates of the other claims.

In [29]:
nonOverlappingClaim :: [FabricClaim] -> Maybe FabricClaim
nonOverlappingClaim xs =
  let precomputedCoords = zip xs $ map getCoords xs
      nonOverlapping :: (FabricClaim, Set.Set (Int, Int)) -> Bool
      nonOverlapping x@(claim, coords) = all (Set.disjoint coords . snd) $ filter (/= x) precomputedCoords
    in
      fst <$> find nonOverlapping precomputedCoords

In [30]:
print . nonOverlappingClaim . map read . lines $ day3Input

Just (FabricClaim {identifier = 1276, position = (568,851), size = (14,23)})

# Day 4

## Part 1

Let's set up some data structures to represent the log data. The log entries in the input look like this:

```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
```

In [31]:
import Data.Time (LocalTime)

type GuardId = Int
data GuardEvent = GuardBeginsShift GuardId | GuardFallsAsleep | GuardWakesUp deriving (Show, Eq, Ord)
data LogEntry = LogEntry LocalTime GuardEvent deriving (Show, Eq, Ord)

Let's see if the `Read` instance of `LocalTime` will work for the format that appears in the input.

In [32]:
read "1518-11-03 00:29" :: LocalTime

Prelude.read: no parse

In [33]:
read "1518-11-03 00:29:00" :: LocalTime

1518-11-03 00:29:00

So it looks like we'll have to append a `":00"` before we try to `read` the timestamp. No big deal. Let's try to parse the log entries.

In [34]:
import Text.ParserCombinators.ReadP (ReadP, char, munch, satisfy, (<++), string)
import qualified Text.ParserCombinators.ReadPrec as ReadPrec
import qualified Text.Read as Read

guardEvent :: ReadP GuardEvent
guardEvent =
  let guardBeginsShift = GuardBeginsShift <$> (string "Guard #" *> (read <$> munch isDigit))
                                          <* string " begins shift"
      guardFallsAsleep = GuardFallsAsleep <$ string "falls asleep"
      guardWakesUp = GuardWakesUp <$ string "wakes up"
    in guardBeginsShift <++ guardFallsAsleep <++ guardWakesUp

instance Read GuardEvent where
  readPrec = ReadPrec.lift guardEvent

instance Read LogEntry where
  readPrec =
    let timestampString = (\x -> read $ x ++ ":00") <$> (char '[' *> munch (/= ']')) <* char ']'
      in ReadPrec.lift $ LogEntry <$> (timestampString <* char ' ') <*> guardEvent

In [35]:
read "[1518-11-01 00:00] Guard #10 begins shift" :: LogEntry

LogEntry 1518-11-01 00:00:00 (GuardBeginsShift 10)

In [36]:
read "[1518-11-01 00:05] falls asleep" :: LogEntry

LogEntry 1518-11-01 00:05:00 GuardFallsAsleep

In [37]:
read "[1518-11-01 00:25] wakes up" :: LogEntry

LogEntry 1518-11-01 00:25:00 GuardWakesUp

Now that we can parse the log entries, we can move on to computing the times that each guard slept.

In [38]:
data OnDutyNap = OnDutyNap GuardId LocalTime LocalTime deriving Show

napper :: OnDutyNap -> GuardId
napper (OnDutyNap guardId _ _) = guardId

import Data.Time.LocalTime (LocalTime(..), TimeOfDay(..))

napDurationInMinutes :: OnDutyNap -> Int
napDurationInMinutes (OnDutyNap _ (LocalTime _ (TimeOfDay _ mStart _)) (LocalTime _ (TimeOfDay _ mEnd _))) =
  mEnd - mStart

In [39]:
napDurationInMinutes $ OnDutyNap 10 (read "1518-11-01 00:05:00") (read "1518-11-01 00:25:00")

20

In [40]:
extractNaps :: [LogEntry] -> [OnDutyNap]
extractNaps =
  let isNapEntry (LogEntry _ GuardFallsAsleep) = True
      isNapEntry (LogEntry _ GuardWakesUp) = True
      isNapEntry _ = False
      getNaps guardId [] = []
      getNaps guardId (LogEntry start GuardFallsAsleep : LogEntry end GuardWakesUp : xs) =
        OnDutyNap guardId start end : getNaps guardId xs
      extract [] = []
      extract (LogEntry _ (GuardBeginsShift guardId):xs) =
        let (napEntries, remainingEntries) = span isNapEntry xs
          in getNaps guardId napEntries ++ extract remainingEntries
    in extract . sort

In [41]:
extractNaps [LogEntry (read "1518-11-01 00:00:00") (GuardBeginsShift 10)
            ,LogEntry (read "1518-11-01 00:05:00") GuardFallsAsleep
            ,LogEntry (read "1518-11-01 00:25:00") GuardWakesUp
            ,LogEntry (read "1518-11-01 00:28:00") GuardFallsAsleep
            ,LogEntry (read "1518-11-01 00:34:00") GuardWakesUp]

[OnDutyNap 10 1518-11-01 00:05:00 1518-11-01 00:25:00,OnDutyNap 10 1518-11-01 00:28:00 1518-11-01 00:34:00]

For part 1, we'll need to get the guard that sleeps the most, and then find the minute that he's asleep for most often.

In [42]:
import Data.List (maximumBy, groupBy, sortBy)
import Data.Ord (comparing)

sleepiestGuard :: [OnDutyNap] -> GuardId
sleepiestGuard =
  let totalMinutesSleeping :: [OnDutyNap] -> Int
      totalMinutesSleeping = sum . map napDurationInMinutes
    in napper . head .
       maximumBy (\a b -> compare (totalMinutesSleeping a) (totalMinutesSleeping b)) .
       groupBy (\a b -> napper a == napper b) .
       sortBy (\a b -> compare (napper a) (napper b))

In [43]:
sleepiestGuard [OnDutyNap 10 (read "1518-11-01 00:05:00") (read "1518-11-01 00:25:00")]

10

In [44]:
sleepiestGuard [OnDutyNap 10 (read "1518-11-01 00:05:00") (read "1518-11-01 00:25:00")
               ,OnDutyNap 11 (read "1518-11-01 00:32:00") (read "1518-11-01 00:55:00")
               ,OnDutyNap 12 (read "1518-11-01 00:01:00") (read "1518-11-01 00:02:00")]

11

In [45]:
minutesSleeping :: OnDutyNap -> [Int]
minutesSleeping (OnDutyNap _ (LocalTime _ (TimeOfDay _ mStart _)) (LocalTime _ (TimeOfDay _ mEnd _))) =
  [mStart..mEnd-1]

In [46]:
minutesSleeping $ OnDutyNap 12 (read "1518-11-01 00:08:00") (read "1518-11-01 00:22:00")

[8,9,10,11,12,13,14,15,16,17,18,19,20,21]

In [47]:
import qualified Data.Ord
import Data.List (sortOn)

mode :: Ord a => [a] -> (Int, a)
mode = head . sortOn Data.Ord.Down . map (\xs -> (length xs, head xs)) . group . sort

mostFrequentMinuteSpentSleeping :: [OnDutyNap] -> (Int, Int)
mostFrequentMinuteSpentSleeping = mode . concatMap minutesSleeping

In [48]:
mostFrequentMinuteSpentSleeping
 [OnDutyNap 10 (read "1518-11-01 00:05:00") (read "1518-11-01 00:25:00")
 ,OnDutyNap 10 (read "1518-11-01 00:30:00") (read "1518-11-01 00:55:00")
 ,OnDutyNap 10 (read "1518-11-01 00:24:00") (read "1518-11-01 00:29:00")]

(2,24)

In [49]:
day4Input <- readFile "./inputs/4.txt"
let entries :: [LogEntry]
    entries = sort . map read . lines $ day4Input
    naps = extractNaps entries
    guardId = sleepiestGuard naps
    (_,minute) = mostFrequentMinuteSpentSleeping (filter ((== guardId) . napper) naps)
  in show guardId ++ " * " ++ show minute ++ " = " ++ show (guardId * minute)

"73 * 44 = 3212"

## Part 2

In [50]:
let entries = sort . map read . lines $ day4Input
    naps = extractNaps entries
    (guardId,(_,minute)) = maximumBy (\(_,a) (_,b) -> compare a b) . 
      map (\xs -> (napper . head $ xs, mostFrequentMinuteSpentSleeping xs)) .
      groupBy (\a b -> napper a == napper b) . sortOn napper $ naps
  in show guardId ++ " * " ++ show minute ++ " = " ++ show (guardId * minute)

"191 * 26 = 4966"

# Day 5

## Part 1

In [51]:
import Data.Bits (xor)
import Data.Char (ord)

type Polymer = (String,String)

toPolymer :: String -> Polymer
toPolymer xs = ([],xs)

fromPolymer :: Polymer -> String
fromPolymer (xs,[]) = reverse xs

reactPolymer :: Polymer -> Polymer
reactPolymer (xs,[]) = (xs,[])
reactPolymer (xs,[y]) = (y:xs,[])
reactPolymer ([],y:z:xs) | (ord y) `xor` 32 == (ord z) = reactPolymer ([], xs)
                         | otherwise = reactPolymer (y:[], z:xs)
reactPolymer (x:xs,y:z:ys) | (ord y) `xor` 32 == (ord z) = reactPolymer (xs, x:ys)
                           | otherwise = reactPolymer (y:x:xs,z:ys)
        
collapseAll :: String -> String
collapseAll = fromPolymer . reactPolymer . toPolymer

In [52]:
collapseAll "dabAcCaCBAcCcaDA"

"dabCBAcaDA"

In [53]:
day5Input <- readFile "./inputs/5.txt"
length . collapseAll . head . lines $ day5Input

11042

## Part 2

In [54]:
import Data.Char (toUpper)
import Data.List (minimumBy)

let units = ['a'..'z']
    originalPolymer = head . lines $ day5Input
    candidates = map (\x -> filter (\y -> y /= x && y /= toUpper x) originalPolymer) units
    reactedCandidates = map collapseAll candidates
    shortest = minimumBy (\a b -> compare (length a) (length b)) reactedCandidates
  in length shortest

6872

# Day 6

Fortunately, parsing the input format is a bit easier for this problem.

In [55]:
import Text.ParserCombinators.ReadP (many, eof, readP_to_S)

listOfCoordinates :: ReadP [(Int, Int)]
listOfCoordinates =
  let coordinate = (\a b -> (read a, read b)) <$> munch isDigit <* string ", " <*> munch isDigit
      in (many $ coordinate <* char '\n') <* eof

In [56]:
day6Input <- readFile "./inputs/6.txt"
let [(parsed,_)] = readP_to_S listOfCoordinates day6Input
  in print parsed

[(45,315),(258,261),(336,208),(160,322),(347,151),(321,243),(232,148),(48,202),(78,161),(307,230),(170,73),(43,73),(74,248),(177,296),(330,266),(314,272),(175,291),(75,142),(278,193),(279,337),(228,46),(211,164),(131,100),(110,338),(336,338),(231,353),(184,213),(300,56),(99,231),(119,159),(180,349),(130,193),(308,107),(140,40),(222,188),(356,44),(73,107),(304,313),(199,238),(344,158),(49,225),(64,117),(145,178),(188,265),(270,215),(48,181),(213,159),(174,311),(114,231),(325,162)]

Let's start off by giving a definition of the manhattan distance between two points.

In [57]:
manhattanDistance :: (Int, Int) -> (Int, Int) -> Int
manhattanDistance (x1, y1) (x2, y2) = abs (x2 - x1) + abs (y2 - y1)

Now we'll want to be able to calculate the closest point out of a list of points. If a point is equally close to two or more points, we'll return `Nothing`.

In [58]:
closestPoint :: [(Int, Int)] -> (Int, Int) -> Maybe (Int, Int)
closestPoint xs x =
  let distances = sortOn snd $ map (\y -> (y, manhattanDistance y x)) xs
      (closest:_) = groupBy (\a b -> snd a == snd b) distances
    in if length closest == 1 then Just (fst . head $ closest) else Nothing

In [59]:
let examplePoints = [(1, 1), (1, 6), (8, 3), (3, 4), (5, 5), (8, 9)]
closestPoint examplePoints (1,1)
closestPoint examplePoints (2,2)
closestPoint examplePoints (3,3)
closestPoint examplePoints (5,1)

Just (1,1)

Just (1,1)

Just (3,4)

Nothing

We want to find the sizes around regions around our given points. My stategy for determining when we're in an "infinite" region will be to expand the group from the given point step-by-step until we generate a coordinate that is not closer to any point than the point that generated it.

In [60]:
data Region = FiniteRegion (Set.Set (Int, Int)) | InfiniteRegion deriving Show

closerToAnyPoint :: [(Int, Int)] -> (Int, Int) -> (Int, Int) -> Bool
closerToAnyPoint xs a b =
  let ds x = map (manhattanDistance x) xs
    in any ((>) 0) $ zipWith (-) (ds a) (ds b)

regionAround :: [(Int, Int)] -> (Int, Int) -> Region
regionAround ps p0 =
  let expand :: Region -> Set.Set (Int, Int) -> (Int, Int) -> (Region, Set.Set (Int, Int))
      expand InfiniteRegion vs p = (InfiniteRegion, Set.insert p vs)
      expand (FiniteRegion rs) vs p@(x,y)
        | p `Set.member` vs || closestPoint ps p /= Just p0 = (FiniteRegion rs, Set.insert p vs)
        | p /= p0 && not (closerToAnyPoint ps p p0) = (InfiniteRegion, Set.insert p vs)
        | otherwise = let (r0, v0) = (FiniteRegion $ Set.insert p rs, Set.insert p vs)
                          (r1, v1) = expand r0 v0 (x + 1, y)
                          (r2, v2) = expand r1 v1 (x - 1, y)
                          (r3, v3) = expand r2 v2 (x, y + 1)
                          (r4, v4) = expand r3 v3 (x, y - 1)
                        in (r4, v4)
    in fst $ expand (FiniteRegion Set.empty) Set.empty p0

regions :: [(Int, Int)] -> [Region]
regions xs = map (regionAround xs) xs

In [61]:
regions examplePoints

[InfiniteRegion,InfiniteRegion,InfiniteRegion,FiniteRegion (fromList [(2,3),(2,4),(3,2),(3,3),(3,4),(3,5),(4,2),(4,3),(4,4)]),FiniteRegion (fromList [(4,5),(4,6),(4,7),(4,8),(5,2),(5,3),(5,4),(5,5),(5,6),(5,7),(5,8),(6,4),(6,5),(6,6),(6,7),(7,5),(7,6)]),InfiniteRegion]

Then, among the finite regions, we want the largest.

In [62]:
regionSize :: Region -> Int
regionSize (FiniteRegion rs) = Set.size rs

isFinite :: Region -> Bool
isFinite InfiniteRegion = False
isFinite (FiniteRegion _) = True

largestRegion :: [Region] -> Region
largestRegion = maximumBy (\a b -> compare (regionSize a) (regionSize b)) . filter isFinite

In [63]:
let [(coords,_)] = readP_to_S listOfCoordinates day6Input
  in print . regionSize . largestRegion . regions $ coords

3894